In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sbs

In [ ]:
test_path = 'datas/test.csv'
train_path = 'datas/train.csv'

test_pd = pd.read_csv(test_path)
train_pd = pd.read_csv(train_path)
train_pd.head(5)

In [ ]:
text2fix = train_pd['input']
textGT = train_pd['output']

max_text2fix_length = max([len(x) for x in text2fix])
max_textGT_length = max([len(x) for x in textGT])

print(max_text2fix_length, max_textGT_length)
# 두 문장 모두 최대길이는 1381 1381

In [ ]:
test_pd.head(5)

In [ ]:
max_test_length = max([len(x) for x in test_pd['input']])
max_test_length, len(test_pd['input'])
# 가장 긴 생성할 테스트 난독화 텍스트 길이 : 1965

In [ ]:
text2reconstruct_string = ''
for strings in text2fix:
    text2reconstruct_string = text2reconstruct_string + strings + '\n'
text2reconstruct_string

In [ ]:
for strings in test_pd['input']:
    text2reconstruct_string = text2reconstruct_string + strings + '\n'
text2reconstruct_string

In [ ]:
# 텍스트를 파일로 저장
file_path = 'datas/text2reconstruct.txt'
with open(file_path, 'w', encoding='utf-8') as file:
    file.write(text2reconstruct_string)

file_path


In [9]:
from tokenizers import Tokenizer, models, trainers, pre_tokenizers, normalizers
from tokenizers.processors import TemplateProcessing

# # 1. WordPiece 모델 생성
# tokenizer = Tokenizer(models.WordPiece(unk_token="<unk>"))

# # 2. 텍스트 정규화 및 전처리 설정
# tokenizer.normalizer = normalizers.Sequence([normalizers.NFKC()])  # 유니코드 정규화
# tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()  # 공백 단위 분리

# # 3. 학습 설정
# trainer = trainers.WordPieceTrainer(
#     vocab_size=50000,  # 어휘 크기 조정 가능
#     special_tokens=["<pad>", "<unk>", "<bos>", "<eos>", "<mask>"]
# )

# # 4. 텍스트 파일 학습 (파일 경로 정확히 설정 필요)
# files = ["datas/text2reconstruct.txt"]
# tokenizer.train(files, trainer)

# # 5. 후처리: 문장 앞뒤에 <bos>, <eos> 토큰 추가
# tokenizer.post_processor = TemplateProcessing(
#     single="<bos> $A <eos>",
#     pair="<bos> $A <eos> <bos> $B:1 <eos>:1",
#     special_tokens=[("<bos>", 1), ("<eos>", 2)]
# )

# # 6. 학습된 토크나이저 저장 및 테스트
# tokenizer.save("wordpiece_tokenizer_50000.json")

# # 토큰화 테스트
# for s in text2fix[:5]:
#     print("원본텍스트:", s)
#     encoded = tokenizer.encode(s)
#     print("토큰화 결과:", encoded.tokens)


In [ ]:
from tokenizers import Tokenizer, models, trainers, pre_tokenizers

# 토크나이저와 BPE 모델 생성
BPE_tokenizer = Tokenizer(models.BPE())

# BPE 트레이너 설정
trainer = trainers.BpeTrainer(special_tokens=["<pad>", "<unk>", "<bos>", "<eos>", "<mask>"], vocab_size=50000)
BPE_tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

# 텍스트 파일 학습
files = ["datas/encoder_augmentation.txt"]  # 정확한 경로로 수정 필요
BPE_tokenizer.train(files, trainer)

# 토큰화 테스트
for s in text2fix[:5]:
    print("원본텍스트:", s)
    encoded = BPE_tokenizer.encode(s)
    print("토큰화 결과:", encoded.tokens)

# 학습된 토크나이저 저장 및 테스트
BPE_tokenizer.save("BPE_tokenizer_50000_aug.json")

In [ ]:
# 파일 경로 지정
file_path = 'datas/text2reconstruct.txt'

# 한 줄씩 읽어서 리스트로 저장
lines = []
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        line = line.strip()  # 앞뒤 공백 제거
        if line:             # 빈 줄은 제외
            lines.append(line)

# 읽어온 결과 출력
print("총 {}줄의 텍스트를 읽어왔습니다.".format(len(lines)))
for i, line in enumerate(lines[:5]):  # 첫 5줄 출력
    print(f"{i + 1}: {line}")


In [ ]:
max_length = 0
for line in lines:  # 첫 5줄 출력
    encoded = tokenizer.encode(line)
    length_encoded = len(encoded.type_ids)
    max_length = length_encoded if length_encoded > max_length else max_length
print(max_length)


In [ ]:
tokenizer.get_vocab_size()

In [17]:
vocab = tokenizer.get_vocab()

In [ ]:
vocab